In [ ]:
#@title # <font class="markdown-google-sans">**<font color="red">1.</font>** Install</font>
import os

!rm -rf "HRAudioWizard"
!git clone "https://github.com/Super-YH/HRAudioWizard"
print()

!pip install -U "git+https://github.com/kubinka0505/axiom#egg=axiom&subdirectory=Files" --use-deprecated=legacy-resolver
print()

!pip install -r "HRAudioWizard/requirements.txt" PyQt6
print()

# fix for pydub
PYDUB_UTILS_FIX_FILE = "https://raw.githubusercontent.com/kubinka0505/colab-notebooks/refs/heads/master/Notebooks/AI/Audio/Upscale/HRAudioWizard/utils.py"

try:
	from flet import FilePickerResultEvent as fpre
except ImportError:
	!pip install flet<=0.19

if os.sys.version_info.minor > 12:
	!pip install audioop-lts

!sudo curl -o /usr/local/lib/python3.10/dist-packages/pydub/utils.py $PYDUB_UTILS_FIX_FILE -s
!sudo curl -o /usr/local/lib/python3.11/dist-packages/pydub/utils.py $PYDUB_UTILS_FIX_FILE -s
!sudo curl -o /usr/local/lib/python3.12/dist-packages/pydub/utils.py $PYDUB_UTILS_FIX_FILE -s

In [ ]:
#@title # <font class="markdown-google-sans">**<font color="gray">2.</font>** Drive connection <font size=4>(optional)</font></font>
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

In [ ]:
#@title # <font class="markdown-google-sans">**<font color=lime>3.</font>** Upscale</font>
import os
import warnings
import numpy as np
from pathlib import Path
from google.colab import files

import soundfile as sf
from HRAudioWizard import (
	HighFrequencyRestorer,
	HighFrequencyRestorerV1,
	RDONoiseShaper,
	IntelligentRDOResampler
)

warnings.filterwarnings("ignore", category = DeprecationWarning)

#@markdown ## <font class="markdown-google-sans">Input files options</font>
Input = "/content/drive/MyDrive/input" #@param {type: "string"}
#@markdown > Can be directory.

Excluded = "_upscaled | _us" #@param {type: "string"}
#@markdown > Pipe-separated values specifying which files to exclude. (file name without extension)
Excluded = list(set(e.strip() for e in Excluded.strip("|").split("|") if e))

Recursive = False #@param {type: "boolean"}

Cutoff_Detection = True #@param {type: "boolean"}

#@markdown > Uses [axiom](https://github.com/kubinka0505/axiom) to detect sample-rate.
#@markdown
#@markdown > ⚠️ <font color=red>**Irreversible, forcefully writes to `Input` files, use with caution.**</font>

#@markdown ## <font class="markdown-google-sans">Output file options</font>
Samplerate = "48 000 Hz" #@param ["32 000 Hz", "44 100 Hz", "48 000 Hz", "88 200 Hz", "96 000 Hz", "192 000 Hz"]
Samplerate = int("".join((c for c in Samplerate if c.isdigit())))

Bit_Depth = "Inherit" #@param ["16-bit (int)", "24-bit (int)", "32-bit (float)", "Inherit"]
Bit_Depth = "".join((c for c in Bit_Depth if c.isdigit()))

Forced = True #@param {type: "boolean"}
#@markdown > Processes in all circumstances, even if $audio_{sr} \le sr$





#@markdown ---
#@markdown ### <font class="markdown-google-sans">Restorer options</font>

Version = "HFPv2" #@param ["HFPv1", "HFPv2"]
Version = int("".join((c for c in "".join(Version.lower().split("v")[1:]) if c.isdigit())))

Compressed_Mode = True #@param {type: "boolean"}

# v1 only

Use_HPSS = True #@param {type: "boolean"}
#@markdown > Use HPSS for harmonic/percussive separation.
#@markdown >
#@markdown > **<font color="gold">Applicable for `v1` only.</font>**

Temporal_Filter_Size = 5 #@param {type: "slider", min: 1, max: 8, step: 1}
#@markdown > Temporal smoothing filter size.
#@markdown >
#@markdown > **<font color="gold">Applicable for `v1` only.</font>**

Griffin_Lim_Iterations = 2 #@param {type: "slider", min: 1, max: 5, step: 1}
#@markdown > **<font color="gold">Applicable for `v1` only.</font>**



#@markdown ## <font class="markdown-google-sans">Resampler options</font>
Chunk_Size = 16384 #@param {type: "slider", min: 512, max: 16384, step: 512}

Filter_Taps = 2048 #@param {type: "slider", min: 128, max: 4096, step: 128}
Filter_Taps -= 1
#@markdown > FIR filter tap count.
#@markdown
#@markdown > Has to be odd, therefore decreased by $1$.

Bands = 128 #@param {type: "slider", min: 64, max: 512, step: 64}
#@markdown > Number of frequency bands in psychoacoustic analysis.

Mid_Side = False #@param {type: "boolean"}
#@markdown > Uses mid-side processing.





#@markdown ---
#@markdown ### <font class="markdown-google-sans">Noise Shaper Options</font>

LPC_Order = 16 #@param {type: "slider", min: 1, max: 32, step: 1}
#@markdown > Predicts the current audio sample from a weighted sum of previous samples.</font>

#-=-=-=-#
# File fetching

Input = Path(Input)
if Input.is_dir():
	Input = list(Input.glob("*" + ("*/*" if Recursive else "")))
else:
	Input = [Input]

Input = [str(file.resolve()) for file in Input if file.is_file()]

if not Input:
	print("No input files found, prompted upload")
	print()
	Input = files.upload()
	Input = list(Input.keys())

for File in Input:
	for Exclusion in Excluded:
		bn = os.path.splitext(os.path.basename(File))[0]
		if bn.endswith(Exclusion):
			Input.remove(File)

if not Input:
	raise Exception("No input files found.")

Outputs = []

#-=-=-=-#
# Preprocessing

Files = []

for File in Input:
	try:
		sf.read(File)
		Files.append(File)
	except Exception as e:
		pass

if Cutoff_Detection:
	for File in Input:
		try:
			!axiom -i $File -o $File -f -v -1
		except Exception as e:
			print(e)
			print("Aborting cutoff detections.")
			break


#-=-=-=-#
# Processing

total = len(Files)
width = len(str(total))

for idx, file in enumerate(Files, 1):
	print(f"[{idx:0{width}d}/{total:0{width}d}] {os.path.relpath(file, os.getcwd())}")

	Output = "_upscaled".join(os.path.splitext(file))

	aObj = sf.SoundFile(file)
	audio = aObj.read()
	sr = aObj.samplerate

	if not Forced:
		if sr <= Samplerate:
			print('Obsolete operation on "{file}", skipping')
			continue

	if audio.ndim == 1:
		audio = np.stack([audio, audio], axis = 1)
	elif audio.shape[1] == 1:
		audio = np.repeat(audio, 2, axis = 1)
	#audio = audio.T # (samples, channels)

	# Restorer
	if Version == 1:
		restorer = HighFrequencyRestorerV1()
		restored = restorer.run_hfp_v1(
			dat = audio,
			sr = sr,

			lowpass = -1,
			compressed_mode = Compressed_Mode,

			use_hpss = Use_HPSS,
			temporal_filter_size = Temporal_Filter_Size,
			griffin_lim_iter = Griffin_Lim_Iterations
		)
	if Version == 2:
		restorer = HighFrequencyRestorer()
		restored = restorer.run_hfpv2(
			dat = audio,
			sr = sr,

			lowpass = -1,

			enable_compressed_fix = Compressed_Mode
		)

	# Resampler
	resampler = IntelligentRDOResampler(
		original_sr = sr,
		target_sr = Samplerate,

		chunk_size = Chunk_Size,
		filter_taps = Filter_Taps,
		num_bands = Bands
	)
	resampled = resampler.resample(restored, Mid_Side)

	# Noise shaper
	if not Bit_Depth:
		Bit_Depth = aObj.subtype.split("_")[-1]

	Bit_Depth = int(Bit_Depth)

	shaper = RDONoiseShaper(
		target_bit_depth = Bit_Depth,
		sr = Samplerate,

		lpc_order = LPC_Order
	)
	output = shaper.process(resampled)

	# Save
	sf.write(Output, output, Samplerate)
	Outputs.append(Output)

	if idx == total:
		print()

#-=-=-=-#

print("Done.")